In [2]:
import pandas as pd

data = pd.read_excel(
    r"C:\Python\projects\CGPAPrediction\data\another_try\data_with_genders\cleaned_data\re_merged.xlsx")

cat_cols = ["ФИО студента", "ОП/Специальность", "Форма оплаты", "Вид финансирования",
            "Язык обучения", "Институт", "Форма обучения", "Группа", "Дисциплина",
            "ФИО преподавателя", "Кафедра преподавателя", "Пол студента", "Пол преподавателя"]

In [3]:
print(len(data['ID студента'].unique()))

2423


In [ ]:
import pandas as pd


def get_cat(series):
    categories = {k: i for i, k in enumerate(series.value_counts().index)}
    return series.map(categories).astype("int64"), categories


def get_cats_mappers(mapping_df):
    direct, reverse = {}, {}
    for col in mapping_df.columns:
        direct[col], reverse[col] = {}, []
        for k, v in mapping_df[col].items():
            if pd.isna(v):
                break
            direct[col][v] = k
            reverse[col].append(v)
    return direct, reverse


def create_mapping(data, cols):
    result, mappings = pd.DataFrame(), {}
    for col in data.columns:
        if col in cols:
            result.loc[:, col], mapping = get_cat(data[col])
            mappings[col] = mapping
        else:
            result.loc[:, col] = data[col]
    return result, mappings

In [4]:
def grade_to_cat(g):
    if g < 70:
        return 0
    elif g < 90:
        return 1
    else:
        return 2

In [5]:
data['cat_grades'] = data['Итоговая оценка'].map(grade_to_cat)

In [6]:
ratios = data['cat_grades'].value_counts()

In [7]:
sm = ratios.sum()
partial_ratios = ratios.map(lambda x: x / sm)

In [8]:
import numpy as np


def count_ratios(df):
    ratios = df['cat_grades'].value_counts()
    sm = ratios.sum()
    return ratios.map(lambda x: x / sm)

def count_max_err(current, balance):
    res = []
    current_ratios = count_ratios(current)
    for i, r in current_ratios.items():
        res.append(abs(r - balance[i]) / balance[i])
    return max(res)


def split_two(df, first_ratio):
    students = df["ID студента"].unique()
    mask = np.zeros(len(students), dtype=bool)
    mask[:int(len(students) * first_ratio)] = True
    np.random.shuffle(mask)
    first_set = set(students[mask])
    first_res_mask = df["ID студента"].isin(first_set)
    return df[first_res_mask], df[~first_res_mask]


def split_two_with_ratio(df, first_ratio, balance_ratio, rel_err):
    firs_df, second_df = split_two(df, first_ratio)
    uns_count = 0
    first_df_me = count_max_err(firs_df, balance_ratio)
    second_df_me = count_max_err(second_df, balance_ratio)
    while first_df_me > rel_err or second_df_me > rel_err:
        print(first_df_me, second_df_me)
        uns_count += 1
        if uns_count > 500:
            print(f'whf? {uns_count} tries')
            raise Exception(f'whf? {uns_count} tries')
        firs_df, second_df = split_two(df, first_ratio)
        first_df_me = count_max_err(firs_df, balance_ratio)
        second_df_me = count_max_err(second_df, balance_ratio)
    return firs_df, second_df


In [9]:
train, other = split_two_with_ratio(data, 0.7, partial_ratios, 0.005)

0.021975896120788724 0.051234886544571506
0.03458888856119443 0.08038392411570253
0.03482621240658094 0.08127414856436387
0.00521038951520918 0.012206729446671854
0.007752443077294948 0.0180819208001712
0.060593500096637005 0.14144219895180235
0.015432450086657373 0.035862522141663335
0.00925054654299744 0.02152582201059157
0.005390185016900883 0.012572152752636524
0.056804292006566066 0.13217424613223813
0.012575564211254173 0.029331444786913435
0.013871750431039932 0.03237657000346242
0.01840434789761749 0.04291340596388929
0.0036489081896265564 0.008500320650932272
0.010748400731735963 0.02508206803289723
0.018217538587596706 0.04283221379526627
0.054999341672152646 0.12843122786304617
0.02755574707613163 0.06385472494308468
0.010675759735105317 0.02488807429883574
0.015478670808472175 0.036135994864709316
0.008191900405289651 0.01908697303474191
0.0411218131761055 0.09600163012355914
0.02784046246182208 0.06481996333142982
0.00925053845009281 0.021557551822423797
0.0356548793650769

In [10]:
val, test = split_two_with_ratio(other, 0.5, partial_ratios, 0.007)

0.013971674764181643 0.021108805801062506
0.05370742075612292 0.04452548775244301
0.010702603192452873 0.010720638046484921
0.041076149186543606 0.03155105786578934
0.018813142491023352 0.019671515289936485
0.0432907054931414 0.05235048297183875
0.045548479735577366 0.05472944541253035
0.03005680104554148 0.02070833849259505
0.06508049034903161 0.05699188391731475
0.05013356101480736 0.059131729784748996
0.08009154584347516 0.07039076653589116
0.065054988466178 0.07362146139927227
0.048115863824550395 0.03888372391563136
0.01733647132379435 0.009746462347586011
0.12550672223212442 0.1340542687187367
0.04668749204201909 0.05555098892757901
0.1229940270626608 0.13201407694518164
0.09587735148377656 0.10415098162863533
0.10184973807193987 0.110502853561995
0.11020862322798938 0.09967157558994287
0.04280037036815307 0.05252476319357297
0.07355979092505192 0.06381744589827062
0.07204653133161618 0.08186396621770779
0.024915390296299644 0.03225052901241865
0.11443749812148717 0.1239221424994

In [11]:
print(count_ratios(train))
print(count_ratios(val))
print(count_ratios(test))

cat_grades
1    0.485847
2    0.430210
0    0.083943
Name: count, dtype: float64
cat_grades
1    0.487696
2    0.427763
0    0.084541
Name: count, dtype: float64
cat_grades
1    0.487578
2    0.427968
0    0.084455
Name: count, dtype: float64


In [12]:
ml = sum(len(a) for a in [train, val, test])
for a in [train, val, test]:
    print(len(a)/ml)

0.6998657752768385
0.15052528329160322
0.14960894143155828


In [13]:
train_m, mapping = create_mapping(train, cat_cols)

In [14]:
def mappings_to_df(mappings):
    res = pd.DataFrame()
    max_len = max(len(m) for m in mappings.values())
    for col_name, mapping in mappings.items():
        res.loc[:, col_name] = list(mapping.keys()) + [None] * (max_len - len(mapping))
    return res

In [15]:
for col in cat_cols:
    val.loc[:, col] = val[col].map(lambda x: mapping[col].setdefault(x, len(mapping[col])))

for col in cat_cols:
    test.loc[:, col] = test[col].map(lambda x: mapping[col].setdefault(x, len(mapping[col])))

In [16]:
to_delete = []
for i, v in val.groupby(["ФИО студента", "ОП/Специальность"]):
    if len(v.groupby("Семестр")) < 5:
        to_delete.append(i)

len(to_delete)

0

In [17]:
for n, i in enumerate([train_m, val, test]):
    tu = set(i['ID студента'].unique())
    for j in range(3):
        if n != j:
            print([train, val, test][j]['ID студента'].isin(tu).sum())

0
0
0
0
0
0


In [18]:
mapping = mappings_to_df(mapping)

In [19]:
train_m.head(1)

,ID студента,ФИО студента,ОП/Специальность,Форма оплаты,Вид финансирования,Язык обучения,Институт,Форма обучения,Группа,Дисциплина,...,ФИО преподавателя,Кафедра преподавателя,Пол студента,Пол преподавателя,РК 1,РК 2,Рейтинг,Экзамен,Итоговая оценка,cat_grades
0,146093,1206,18,0,0,0,3,3,251,1311,...,137,59,0,0,94.0,95,95,96.0,95,2


In [21]:
columns_translation = {
    "ID студента": "Student ID",
    "ФИО студента": "Student Full Name",
    "ОП/Специальность": "Educational Program / Major",
    "Форма оплаты": "Payment Type",
    "Вид финансирования": "Funding Type",
    "Язык обучения": "Language of Instruction",
    "Институт": "Institute",
    "Форма обучения": "Study Mode",
    "Группа": "Group",
    "Дисциплина": "Course",
    "Семестр": "Semester",
    "ФИО преподавателя": "Instructor Full Name",
    "Кафедра преподавателя": "Instructor Department",
    "Пол студента": "Student Gender",
    "Пол преподавателя": "Instructor Gender",
    "РК 1": "Midterm 1",
    "РК 2": "Midterm 2",
    "Рейтинг": "Rating",
    "Экзамен": "Exam",
    "Итоговая оценка": "Final Grade"
}



In [22]:
dfs = {'train': train_m, 'val':val, 'test': test, 'mapping': mapping}
list(map(lambda x: x.rename(columns=columns_translation, inplace=True), dfs.values()))

C:\Users\Михаил\AppData\Local\Temp\ipykernel_17548\905414056.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list(map(lambda x: x.rename(columns=columns_translation, inplace=True), dfs.values()))


[None, None, None, None]

In [23]:
for df in dfs.values():
    if 'ФИО студента' in df.columns:
        df.drop('ФИО студента', axis=1, inplace=True)
    if 'cat_grades' in df.columns:
        df.drop('cat_grades', axis=1, inplace=True)

C:\Users\Михаил\AppData\Local\Temp\ipykernel_17548\3134429695.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('cat_grades', axis=1, inplace=True)
C:\Users\Михаил\AppData\Local\Temp\ipykernel_17548\3134429695.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('cat_grades', axis=1, inplace=True)


In [24]:
output_train = r"C:\Python\projects\CGPAPrediction\data\final_grade_prediction_dataset\train.csv"
output_val = r"C:\Python\projects\CGPAPrediction\data\final_grade_prediction_dataset\val.csv"
output_test = r"C:\Python\projects\CGPAPrediction\data\final_grade_prediction_dataset\test.csv"
output_mapping = r"C:\Python\projects\CGPAPrediction\data\final_grade_prediction_dataset\mapping.csv"
dfs['train'].to_csv(output_train, index=False)
dfs['val'].to_csv(output_val, index=False)
dfs['test'].to_csv(output_test, index=False)
dfs['mapping'].to_csv(output_mapping)

In [1]:
output_mapping_1 = r"C:\Python\projects\CGPAPrediction\data\final_grade_prediction_dataset\mapping_1.csv"
dfs['mapping'].to_csv(output_mapping_1, encoding="utf-8-sig")

NameError: name 'dfs' is not defined